In [14]:
import sys  
sys.path.insert(0, '..')

from datetime import date, timedelta

from config import Config

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

db = create_engine(Config.SQLALCHEMY_DATABASE_URI)

In [34]:
capacities = pd.read_sql_query("select * from kapacity_nemocnic", db.engine).fillna(0)
capacities['luzka_standard_kyslik_kapacita_volna_covid_pozitivni'] += capacities['inf_luzka_kyslik_kapacita_volna_covid_pozitivni']
capacities['luzka_standard_kyslik_kapacita_volna_covid_negativni'] += capacities['inf_luzka_kyslik_kapacita_volna_covid_negativni']
capacities['luzka_standard_kyslik_kapacita_celkem'] += capacities['inf_luzka_kyslik_kapacita_celkem']
capacities['luzka_hfno_cpap_kapacita_volna_covid_pozitivni'] += capacities['inf_luzka_hfno_kapacita_volna_covid_pozitivni']
capacities['luzka_hfno_cpap_kapacita_volna_covid_negativni'] += capacities['inf_luzka_hfno_kapacita_volna_covid_negativni']
capacities['luzka_hfno_cpap_kapacita_celkem'] += capacities['inf_luzka_hfno_kapacita_celkem']
capacities['luzka_upv_niv_kapacita_volna_covid_pozitivni'] += capacities['inf_luzka_upv_kapacita_volna_covid_pozitivni']
capacities['luzka_upv_niv_kapacita_volna_covid_negativni'] += capacities['inf_luzka_upv_kapacita_volna_covid_negativni']
capacities['luzka_upv_niv_kapacita_celkem'] += capacities['inf_luzka_upv_kapacita_celkem']

capacities = capacities.drop(columns=['inf_luzka_kyslik_kapacita_volna_covid_pozitivni', 'inf_luzka_kyslik_kapacita_volna_covid_negativni', 'inf_luzka_kyslik_kapacita_celkem',
                                      'inf_luzka_hfno_kapacita_volna_covid_pozitivni', 'inf_luzka_hfno_kapacita_volna_covid_negativni', 'inf_luzka_hfno_kapacita_celkem',
                                      'inf_luzka_upv_kapacita_volna_covid_pozitivni', 'inf_luzka_upv_kapacita_volna_covid_negativni', 'inf_luzka_upv_kapacita_celkem'])

# capacities

In [35]:
capacities_old = pd.read_sql_query("select * from kapacity_nemocnic_stare", db.engine).fillna(0)
# capacities_old

In [37]:
kraje = pd.read_sql_query("select * from kraje", db.engine)
kraje['nazev_norm'] = kraje['nazev'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
# kraje

In [39]:
df = pd.concat([capacities_old, capacities])

df = pd.merge(df, kraje, left_on='kraj_nuts_kod', right_on='id')
df = df.drop(columns=['zz_kod', 'zz_nazev', 'id'])
df = df.groupby(['kraj_nuts_kod', 'datum', 'nazev', 'nazev_norm']).sum().reset_index()

df_cr = df.groupby('datum').sum().reset_index()
df_cr['kraj_nuts_kod'] = 'CZ0'
df_cr['nazev'] = 'ČR'
df_cr['nazev_norm'] = 'cr'

df = df.set_index(['nazev_norm', 'datum'])
df_cr = df_cr.set_index(['nazev_norm', 'datum'])

df = pd.concat([df_cr, df])

df['ventilatory_volno'] = df['ventilatory_prenosne_kapacita_volna'] + df['ventilatory_operacni_sal_kapacita_volna']
df['ventilatory_celkem'] = df['ventilatory_prenosne_kapacita_celkem'] + df['ventilatory_operacni_sal_kapacita_celkem']

df = df.drop(columns=['ventilatory_prenosne_kapacita_volna', 'ventilatory_operacni_sal_kapacita_volna', 
                      'ventilatory_prenosne_kapacita_celkem', 'ventilatory_operacni_sal_kapacita_celkem'])

df = df.rename(columns={'luzka_standard_kyslik_kapacita_volna_covid_pozitivni': 'kyslik_volno_poz', 
                        'luzka_standard_kyslik_kapacita_volna_covid_negativni': 'kyslik_volno_neg', 
                        'luzka_standard_kyslik_kapacita_celkem': 'kyslik_celkem', 
                        'luzka_hfno_cpap_kapacita_volna_covid_pozitivni': 'hfno_volno_poz', 
                        'luzka_hfno_cpap_kapacita_volna_covid_negativni': 'hfno_volno_neg', 
                        'luzka_hfno_cpap_kapacita_celkem': 'hfno_celkem', 
                        'luzka_upv_niv_kapacita_volna_covid_pozitivni': 'upv_volno_poz', 
                        'luzka_upv_niv_kapacita_volna_covid_negativni': 'upv_volno_neg',
                        'luzka_upv_niv_kapacita_celkem': 'upv_celkem', 
                        'ecmo_kapacita_volna': 'ecmo_volno', 
                        'ecmo_kapacita_celkem': 'ecmo_celkem', 
                        'cvvhd_kapacita_volna': 'cvvhd_volno', 
                        'cvvhd_kapacita_celkem': 'cvvhd_kapacita'})
df



kyslik_volno_poz  kyslik_volno_neg  kyslik_celkem  \
nazev_norm      datum                                                           
cr              2021-03-25               166               288           2193   
                2021-03-26              1794              3152          18660   
                2021-03-27              1970              3607          21276   
                2021-03-28              1806              3655          21348   
                2021-03-29              1792              3758          22021   
...                                      ...               ...            ...   
moravskoslezsky 2021-11-22               138               517           2964   
                2021-11-23               128               461           2964   
                2021-11-24               129               455           2964   
                2021-11-25               153               416           2964   
                2021-11-26               175               417           2964   

                            hfno_volno_poz  hfno_volno_neg  hfno_celkem  \
nazev_norm      datum                                                     
cr              2021-03-25              39               7          188   
                2021-03-26             463             170         1549   
                2021-03-27             486             194         1801   
                2021-03-28             489             208         1796   
                2021-03-29             496             190         1818   
...                                    ...             ...          ...   
moravskoslezsky 2021-11-22              64              36          212   
                2021-11-23              59              36          212   
                2021-11-24              53              37          212   
                2021-11-25              52              35          212   
                2021-11-26              55              34          212   

                            upv_volno_poz  upv_volno_neg  upv_celkem  \
nazev_norm      datum                                                  
cr              2021-03-25             16             22         166   
                2021-03-26            121            204        2273   
                2021-03-27            144            244        2210   
                2021-03-28            129            254        2233   
                2021-03-29            129            217        2285   
...                                   ...            ...         ...   
moravskoslezsky 2021-11-22             22             32         311   
                2021-11-23             21             41         311   
                2021-11-24             25             37         311   
                2021-11-25             28             31         311   
                2021-11-26             24             28         311   

                            ecmo_volno  ecmo_celkem  cvvhd_volno  \
nazev_norm      datum                                              
cr              2021-03-25           3           21           16   
                2021-03-26          22           83          150   
                2021-03-27          28           93          163   
                2021-03-28          27           93          166   
                2021-03-29          28           93          165   
...                                ...          ...          ...   
moravskoslezsky 2021-11-22           5           11           24   
                2021-11-23           4           11           24   
                2021-11-24           4           11           24   
                2021-11-25           4           11           24   
                2021-11-26           4           11           24   

                            cvvhd_kapacita kraj_nuts_kod            nazev  \
nazev_norm      datum                                                       
cr              2021-0

In [43]:
df[df.index.get_level_values('nazev_norm') == 'cr'].droplevel('nazev_norm')

,kyslik_volno_poz,kyslik_volno_neg,kyslik_celkem,hfno_volno_poz,hfno_volno_neg,hfno_celkem,upv_volno_poz,upv_volno_neg,upv_celkem,ecmo_volno,ecmo_celkem,cvvhd_volno,cvvhd_kapacita,kraj_nuts_kod,nazev,ventilatory_volno,ventilatory_celkem
datum,,,,,,,,,,,,,,,,,
2021-03-25,166,288,2193,39,7,188,16,22,166,3,21,16,44,CZ0,ČR,122,163
2021-03-26,1794,3152,18660,463,170,1549,121,204,2273,22,83,150,263,CZ0,ČR,1114,1448
2021-03-27,1970,3607,21276,486,194,1801,144,244,2210,28,93,163,283,CZ0,ČR,1173,1590
2021-03-28,1806,3655,21348,489,208,1796,129,254,2233,27,93,166,286,CZ0,ČR,1179,1537
2021-03-29,1792,3758,22021,496,190,1818,129,217,2285,28,93,165,289,CZ0,ČR,1201,1556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-22,1138,4799,23884,366,337,1792,154,286,2088,54,89,222,306,CZ0,ČR,1250,1555
2021-11-23,1201,4398,23870,372,319,1794,151,285,2090,54,89,230,306,CZ0,ČR,1238,1555
2021-11-24,1164,4291,23912,340,309,1794,162,279,2091,53,89,231,306,CZ0,ČR,1240,1555
